In [1]:
import os
import sys
import math
import cv2
import imutils
import numpy as np
import pandas as pd
from keras.models import Model
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.models import model_from_json
from tensorflow.keras.backend import resize_images

Using TensorFlow backend.
C:\Users\Naman\anaconda3\envs\cv2_tf\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Naman\anaconda3\envs\cv2_tf\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Naman\anaconda3\envs\cv2_tf\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Naman\anaconda3\envs\cv2_tf\lib\sit

## Load Model and use for prediction

In [4]:
json_file = open('Models/model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
# load weights into new model
model.load_weights("Models/model.h5")
print("Loaded model from disk")

# evaluate loaded model on test data
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


Loaded model from disk


In [5]:
# min_YCrCb = np.array([0,133,77],np.uint8)
# max_YCrCb = np.array([255,173,127],np.uint8)

# def extractSkin(image):
#     # Taking a copy of the image
#     img = image.copy()
#     # Converting from BGR Colours Space to HSV
#     img = cv2.cvtColor(image,cv2.COLOR_BGR2YCR_CB)

#     # Single Channel mask,denoting presence of colours in the about threshold
#     skinMask = cv2.inRange(img,min_YCrCb,max_YCrCb)
#     contours, hierarchy = cv2.findContours(skinMask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

#     # Cleaning up mask using Gaussian Filter
# #     skinMask = cv2.GaussianBlur(skinMask, (3, 3), 0)
#     for i, c in enumerate(contours):
#         area = cv2.contourArea(c)
#         if area > 1000:
#             cv2.drawContours(image, contours, i, (0, 255, 0), 3)

#     # Extracting skin from the threshold mask
# #     skin = cv2.bitwise_and(img, img, mask=skinMask)

#     # Return the Skin image
#     return image

In [6]:
videoCaptureObject = cv2.VideoCapture(0)
i = 0
ret = True

# face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
# eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')

# img = cv2.imread('sachin.jpg')
# gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)


fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.avi',fourcc, 20.0, (480,640))
start_point = (50,100)
end_point = (300, 350) 
c = 0;
while(ret):
    ret,frame = videoCaptureObject.read()
    if ret == True:
        clone = frame.copy() 
        clone = clone[...,::-1]
        frame = cv2.rectangle(frame, start_point, end_point, color=(0,0,255), thickness=2)
        
        if(c%2==0):
            ## All the processing has to be done here.
            crop_img = clone[100:350,50:300]
#             img = extractSkin(crop_img)
#             frame[100:350,50:300] = img
            resized_img = cv2.resize(crop_img,(60,60))
#             print(resized_img.shape)

            y_pred = model.predict(resized_img.reshape((1,60,60,3)), batch_size=None, steps=1)
            display_text = ""
            max_pred = np.max(y_pred)
            if(max_pred>0.5):
                display_text = "Alphabet - " + chr(ord('A') + np.argmax(y_pred))
#         # before feeding into imshow()
            
        c += 1
        c = c%2
#         print(frame.shape)
        frame = cv2.flip(frame,1)
        cv2.putText(frame, display_text, (340, 95), cv2.FONT_HERSHEY_SIMPLEX, 1,(0,0,255), 3)
        cv2.imshow('ASL Alphabet Classification', frame)
        # doesn't work with colab (cv2.imshow)
        
        
        out.write(frame)
        pressedKey = cv2.waitKey(1) & 0xFF
        if(pressedKey == ord('c')):
            
            cv2.imwrite("A/A"+ str(i)+".jpg",resized_img)
            plt.imshow(resized_img, vmin=0, vmax=255)
            plt.show()
            i += 1
        elif(pressedKey == ord('q')):
            videoCaptureObject.release()
            out.release()
            cv2.destroyAllWindows()
    else:
        print("Happy Pictures Taken!")


Happy Pictures Taken!


## Generate Your Own data set


In [27]:
# Declare Some variables here
i = 4000
start_point = (50,100)
end_point = (300, 350) 

In [26]:
videoCaptureObject = cv2.VideoCapture(0)
ret = True
while(ret):
    ret,frame = videoCaptureObject.read()
    if ret == True:
        clone = frame.copy() 
        frame = cv2.rectangle(frame, start_point, end_point, color=(0,0,255), thickness=2)
        
        ## All the processing has to be done here.
        crop_img = clone[100:350,50:300]
        resized_img = cv2.resize(crop_img,(100,100))
        ## before feeding into imshow()

        frame = cv2.flip(frame,1)
        cv2.imshow('ASL Alphabet Classification', frame)  
                
        pressedKey = cv2.waitKey(1) & 0xFF
        if (pressedKey >= ord('a')) & (pressedKey <= ord('z')):
            ch = chr(pressedKey - ord('a') + ord('A'))
            path1 = "naman_data/" + ch + "/" + ch + str(i) +".jpg"
            path2 = "naman_data/" + ch + "/" + ch + str(i+1) +".jpg"
            print(path1)
            print(path2)
            cv2.imwrite(path1, resized_img)
            cv2.imwrite(path2, cv2.flip(resized_img,1))
            i += 2
        elif pressedKey == ord('1'):
            videoCaptureObject.release()
            cv2.destroyAllWindows()
    else:
        print("Dataset Generated")

naman_data/A/A2000.jpg
naman_data/A/A2001.jpg
naman_data/A/A2002.jpg
naman_data/A/A2003.jpg
naman_data/A/A2004.jpg
naman_data/A/A2005.jpg
naman_data/A/A2006.jpg
naman_data/A/A2007.jpg
naman_data/A/A2008.jpg
naman_data/A/A2009.jpg
naman_data/A/A2010.jpg
naman_data/A/A2011.jpg
naman_data/A/A2012.jpg
naman_data/A/A2013.jpg
naman_data/A/A2014.jpg
naman_data/A/A2015.jpg
naman_data/A/A2016.jpg
naman_data/A/A2017.jpg
naman_data/A/A2018.jpg
naman_data/A/A2019.jpg
naman_data/A/A2020.jpg
naman_data/A/A2021.jpg
naman_data/A/A2022.jpg
naman_data/A/A2023.jpg
naman_data/A/A2024.jpg
naman_data/A/A2025.jpg
naman_data/A/A2026.jpg
naman_data/A/A2027.jpg
naman_data/A/A2028.jpg
naman_data/A/A2029.jpg
naman_data/A/A2030.jpg
naman_data/A/A2031.jpg
naman_data/A/A2032.jpg
naman_data/A/A2033.jpg
naman_data/A/A2034.jpg
naman_data/A/A2035.jpg
naman_data/A/A2036.jpg
naman_data/A/A2037.jpg
naman_data/A/A2038.jpg
naman_data/A/A2039.jpg
naman_data/A/A2040.jpg
naman_data/A/A2041.jpg
naman_data/A/A2042.jpg
naman_data/